In [2]:
import sys
sys.path.append('../')

In [3]:
from model_applier import ModelApplier
from thesaurus_parsing.thesaurus_parser import ThesaurusParser
import numpy as np
import json
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook as tqdm
from abc import ABCMeta, abstractmethod

### Разбиение на обучение и тест

In [4]:
thesaurus = ThesaurusParser("../data/RuThes", need_closure=False)

Посмотрим, сколько всего есть списков гиперонимов в тезаурусе

In [5]:
len(thesaurus.hypernyms_dict)

110163

Выделим $20$ процентов из этих списков гиперонимов для оценивания моделей. Остальные объявим обучающей выборкой, будем на ней учить модель.

In [6]:
hyponym_keys = list(thesaurus.hypernyms_dict.keys())
train_keys, test_keys = train_test_split(hyponym_keys, test_size=0.2, shuffle=True)

Запишем в файл то, какие получились ключи, чтобы зафиксировать это разделение

In [7]:
with open('train_keys', 'w') as train_file:
    train_file.write('\n'.join(train_keys))
    
with open('test_keys', 'w') as test_file:
    test_file.write('\n'.join(test_keys))

### Оценка модели

Требуется написать класс модели, который умеет обучиться, примениться и соответственно посчитать метрики по тестовому датасету. Пока можно просто написать отдельные функции из него.

Например, требуется для каждого запроса-гипонима применить модель к каждой паре этого гипонима с кандидатами. Это довольно долго, но можно проходить батчами.

Такую логику и надо реализовать

In [8]:
from models import CRIMModel, make_model_vocab
import torch
from collections import defaultdict
from abc import ABC, abstractmethod

In [9]:
model = CRIMModel(n_matrices=24)

In [10]:
model.load_state_dict(torch.load('../data/models/projection_model.bin'))

<All keys matched successfully>

In [11]:
vocab = make_model_vocab()

In [12]:
def apply_model_to_query(query, model, vocab, batch_size=None):
    query_emb = torch.FloatTensor(vocab[query]).unsqueeze(0)
    candidate_batch = []
    
    if batch_size is None:
        batch_size = len(vocab)
    
    for candidate, embedding in vocab.items():
        candidate_batch.append(torch.FloatTensor(embedding).unsqueeze(0))
        if len(candidate_batch) == batch_size:
            candidate_batch = torch.cat(candidate_batch)
            model_batch = {
                'batch': query_emb,
                'candidate': candidate_batch
            }
            probas = model(model_batch)
            candidate_batch = []

In [13]:
for query in tqdm(test_keys):
    apply_model_to_query(query, model, vocab)

KeyboardInterrupt: 

In [13]:
with open('vocab.json', 'w') as vocab_file:
    json.dump({w: e.tolist() for w, e in vocab.items()}, vocab_file)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/loginov-ra/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-6b91348e7cab>", line 2, in <module>
    json.dump({w: e.tolist() for w, e in vocab.items()}, vocab_file)
  File "/usr/lib/python3.6/json/__init__.py", line 179, in dump
    for chunk in iterable:
  File "/usr/lib/python3.6/json/encoder.py", line 430, in _iterencode
    yield from _iterencode_dict(o, _current_indent_level)
  File "/usr/lib/python3.6/json/encoder.py", line 404, in _iterencode_dict
    yield from chunks
  File "/usr/lib/python3.6/json/encoder.py", line 301, in _iterencode_list
    if isinstance(value, str):
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/loginov-ra/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2040, in showtrac

KeyboardInterrupt: 

In [14]:
class DumbApplier(ModelApplier):
    def __init__(self, vocab):
        super().__init__(vocab)
        
    def apply_model_to_query(self, query):
        return np.arange(len(self.vocab))

In [15]:
correct_answers = dict()
for test_key in test_keys:
    correct_answers[test_key] = thesaurus.hypernyms_dict[test_key]

In [16]:
applier = DumbApplier(vocab)
applier.load_correct_answers(correct_answers)

In [17]:
applier._leave_top_apply_to_query('путин', 10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [18]:
applier.apply(test_keys)

In [19]:
applier.get_map()

6.642680762904668e-05